# In this notebook, we are qualitatively building categories

We look at the file: \
`../data/06-clustered-df/alpha0.3_k10_res0.002.pkl`

And the corresponding excel file: \
`..output/tables/cluster-explorer/FinalSelect/SingleSolExplorer_alpha0.3_k10_res0.002.xlsx`


## Trees:

- Pharmacology

  - Pharmacodynamics
    - Mechanism of action
    - Other
  - Pharmacokinetics
  - Pharmacogenetics
  - Other

- Indications

  - Disorders
    - Depression
    - Anxiety
    - OCD
    - Other
  - Special Popultions
  - Other

- Safety

  - Adverse effects
  - Contraindications
  - Drug Interactions
  - Other

- Other
  - Drugs
  - Ecotoxicology
  - Other

### Papers will be categorized into the categories above and can reveive the tags:

1. Clinical Evidence
2. In vivo / In vitro
3. Review
4. Irrelevant


# load and prep data


In [2]:
import pandas as pd
import json

In [3]:
params = "alpha0.3_k10_res0.002"
p = f"../data/06-clustered-df/{params}.pkl"
df = pd.read_pickle(p)
df.head(3)

,eid,title,date,first_author,abstract,doi,year,auth_year,unique_auth_year,pubmed_id,...,reference_eids,nr_references,filtered_reference_eids,nr_filtered_references,title_abstract,clean_title,clean_abstract,specter2_embeddings,cluster_alpha0.3_k10_res0.002,centrality_alpha0.3_k10_res0.002
0,2-s2.0-0020316326,Serotonergic mechanism in the control of β-end...,1982-04-12,Bruni J.F.,The role of the serotonergic mechanism in the ...,10.1016/0024-3205(82)90686-5,1982,Bruni_1982,Bruni_1982,6283286.0,...,"[2-s2.0-0016795422, 2-s2.0-0000011578, 2-s2.0-...",46,[],0,Serotonergic mechanism in the control of β-end...,Serotonergic mechanism in the control of β-end...,The role of the serotonergic mechanism in the ...,"[-0.38758993, 0.8743463, -0.52714413, 0.029653...",70,0.045627
1,2-s2.0-0019936013,EFFECTS OF PAROXETINE ON SYNAPTOSOMAL NEUROTRA...,1982-01-01,Magnussen I.,NaN,10.1111/j.1600-0404.1982.tb03382.x,1982,Magnussen_1982,Magnussen_1982_4,NaN,...,[],0,[],0,EFFECTS OF PAROXETINE ON SYNAPTOSOMAL NEUROTRA...,EFFECTS OF PAROXETINE ON SYNAPTOSOMAL NEUROTRA...,,"[0.329068, 0.23448052, -0.6597941, 0.13635367,...",65,0.010706
2,2-s2.0-0020058010,Treatment of intention myoclonus with paroxeti...,1982-01-01,Magnussen I.,NaN,NaN,1982,Magnussen_1982,Magnussen_1982_3,NaN,...,[],0,[],0,Treatment of intention myoclonus with paroxeti...,Treatment of intention myoclonus with paroxeti...,,"[0.406605, 1.0992043, -0.60125256, 0.73224956,...",42,0.038800


# helper func


In [7]:
def cluster_title_print(df,  cluster, random=False, n=10, random_state=1887):
    """
    Print titles of documents belonging to a specific cluster.

    Args:
    - df: DataFrame containing document data.
    - cluster: Cluster number for which titles are to be printed.
    - random: Whether to select titles randomly. Defaults to False.
    - n: Number of titles to print. Defaults to 10.
    - random_state: Seed for random sampling. Defaults to None.

    Returns:
    - None
    """
    # Filter DataFrame for the specified cluster
    cluster_df = df[df["cluster_alpha0.3_k10_res0.002"] == cluster]
    
    # Sort DataFrame by centrality
    cluster_df = cluster_df.sort_values(by="centrality_alpha0.3_k10_res0.002", ascending=False)
    
    # Select titles
    if random:
        titles = cluster_df.sample(n, random_state=random_state)['title'].values
    else:
        titles = cluster_df.head(n)['title'].values
    
    # Print titles
    print(f"Cluster {cluster}:")
    for title in titles:
        print(title)
        print("# " * 30)
    print("\n")

cluster_title_print(df= df,cluster= 150, random=True, n=3, random_state=None)

Cluster 10:
Effects of repeated phencyclidine treatment on serotonin transporter in rat brain
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
[<sup>123</sup>I]FP-CIT binding in rat brain after acute and sub-chronic administration of dopaminergic medication
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
Measurement of changes in endogenous serotonin level by positron emission tomography with [<sup>18</sup>F]altanserin
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 




# Clusters labels are arrived at by assessing the file:

`Study-1-Bibliometrics/output/tables/cluster-explorer/FinalSelect/SingleSolExplorer_alpha0.3_k10_res0.002.xlsx`

The file contains the 20 most central papers in each cluster as well as 20 random papers from each cluster. 0. What is the defining characteristic of the cluster? (to use as label)

1. Does it focus on pharmacology?
   - Which area of pharmacology?
2. Does it focus on in vivo, vitro, or human?
3. Does it focus on a specific disorder?
4. Does it focus on a special population?
5. Does it focus on a specific drug?
6. Does it focus on safety?

then it gets one of the following tags.

- 1 clinical
- 2 in vivo/vitro
- 3 mixed
- 4 review
- 5 garbage
- 0 None.

The file is manually assessed and the clusters are labeled as follows:

A cleaner version in JSON format is saved as `output/cluster-qualifications/K10/ClusterLabels.json`


In [5]:
# 1 clinical
# 2 in vivo/vitro 
# 3 Mixed
# 4 review
# 5 garbage
# 0 None.

cluster_labels = {
    0: ["Serotonin Receptor Studies", 2], 
    1: ["Aquatic Ecotoxicology", 0], 
    2: ["Prenatal Exposure",1 ], 
    3: ["quantification of SSRIs in biological samples", 2], 
    4: ["Treament of Obsessive-Compulsive Disorder (OCD)", 1], 
    5: ["SSRIs and the Cytochrome P450 System", 2], 
    6: ["Neurogenesis",2 ], 
    7: ["Pediatric Depression", 1], 
    8: ["Herbal and Natural Compounds for Depression, using Forces Swimming Test", 2], # or forced swimming test? this vs 35. here precl. the other clinical
    9: ["Paroxetine Binding", 2 ], 
    10: ["Neural Processing of Emotional Cues", 1],
    11: ["Prenatal Exposure",2],
    12: ["forced swimming test", 2],
    13: ["serotonin transporter gene and antidepressant response", 1], # is this clinical? pharmacogenetics..
    14: ["Fluvoxamine for Depression",2 ],
    15: ["SSRIs for PTSD", 2 ],
    16: ["Sexual Dysfunction",1 ],
    17: ["Sequenced Depression Treatment", 1],
    18: ["Serotonin Syndrome",1 ],
    19: ["Serotonin and Inflammation", 3], # this seems like ssri have anti-inflammatory effects but also inflammation can cause depression
    20: ["Post-Stroke SSRI Use", 1],
    21: ["depression and cardiovascular pathology",1 ], # quick check pls! its rather depression in this population
    22: ["SSRIs for Pain",1 ],
    23: ["SSRIs for Bipolar Depression", 1],
    24: ["SSRI Utilization Patterns", 1],
    25: ["Escitalopram for Depression",1 ],
    26: ["SSRIs in Dementias", 1], # for depression and other symptoms
    27: ["Bleeding Risk", 1],
    28: ["Serotonin Receptor Studies", 2], # compare to 0
    29: ["SSRIs for Weight Management", 1],
    30: ["Suicice Risk",1 ],
    31: ["Cost-Effectiveness",0 ], 
    32: ["Conditioned Fear",2 ], 
    33: ["Paroxetine",4,5 ], 
    34: ["SSRIs for Panic Disorder", 1], 
    35: ["Herbal and Natural Compounds for Depression" , 2], 
    36: ["Fluvoxamine for Covid 19", 1 ], 
    37: ["SSRIs for Generalized Anxiety Disorder (GAD)", 1], 
    38: ["Fracture Risk",1 ], 
    39: ["Fluoxetine", 4,5 ], 
    40: ["Extrapyramidal Reactions Risk", 1], # movement disorders
    41: ["SSRIs Effect on Sleep in Depressed Patients", 1 ], 
    42: ["Hot Flushes (Menopause)", 1], 
    43: ["SSRIs for Dermatological Diseases", 1], # psychodermatology? 
    44: ["Oncopreventive and oncolytic properties of SSRIs",1 ], # mostly risk estimation 
    45: ["Premature Ejaculation",1], 
    46: ["QTC Prolongation", 1], # thats cardiovascular, right?
    47: ["SSRIs for Premenstrual Dysphoric Disorder (PMDD)",1 ], 
    48: ["Depression in Diabetes", 1], 
    49: ["SSRIs for Depression with Cognitive Impairment",1 ], 
    50: ["SSRIs for Eating Disorders", 1], 
    51 : ["SSRIs for Alcohol Dependence",1 ],
    52 : ["SSRIs for Irritable Bowel Disease", 1],
    53 : ["Personality Disorder; Learning; Behavioral Effects; introception", 4], # most central are all borderline personality disorder. random ones are very weird
    54: ["Depression Treatment Review", 1 , 4 ],
    55: ["Depression in Epileptic Patients", 1],
    56: ["SSRIs for Autism Spectrum Disorders",1 ],
    57: ["(Sertraline) Drug Delivery",2],
    58: ["Depression in Parkinsons Patients", ],
    59: ["Hepatotoxic Risk", 2 ], # also check 87 and hepatitis
    60: ["SSRI Synthesis", 2],
    61: ["SSRIs (Fluvoxamine) for Schizophrenia",1,],
    62: ["SSRIs for Substance Abuse (Cocaine)", 2 ],
    63: ["SSRIs for Social Anxiety Disorder",1,4 ],
    64: ["Sertraline for Depression",1 ],
    65: ["Drug Drug Interactions",1 ],
    66: ["Ion Channels", 2],
    67: ["Methylenedioxymethamphetamine (MDMA) Effects on Serotonin", 2],
    68: ["Genotype Mediated Response to SSRIs", 2],
    69: ["Hyponatremia Risk", 1],
    70: ["Neuroendocrine Effects of Citalopram", 3],
    71: ["SSRIs for Body Dysmorphic Disorder (BDD)", 1],
    72: ["Pharmacological Reviews",2,4 ],
    73: ["Forensic Drug Concentration", 1],
    74: ["Citalopram for Depression",1,4 ],
    75: ["SSRIs for Pediatric Anxiety", ],
    76: ["SSRI Safety Reviews",1,4 ],
    77: ["Pulmonary Hypertension", 2],
    78: ["SSRIs as Antifungal Agents",2 ], # where to put this ? dynamics? disorder? Both?
    79: ["Geriatric Depression", 1,4],
    80: ["Withdrawal Symptoms",1 ],
    81: ["Agomelatin for Depression", 4 ],
    82: ["SSRI Efficacy in Depression",1,4 ],
    83: ["Wastewater Management", 0],
    84: ["Astrocyte Receptors",1 ],
    85: ["Sertraline", 5],
    86: ["Vilazodone for Depression",1 ],
    87: ["Depression in Hepatitis Patients", 1],
    88: ["SSRIs for Depression after Traumatic Brain Injury", ],
    89: ["SSRIs Effects on Dopamine Receptors",2 ],
    90: ["Hyperprolactinemia-related Symptoms", 2], # galactorrhoea; amenorrhea; gynecomastia
    91: ["SSRIs for Fibromyalgia", 2], # also chronic fatigue syndrome
    92: ["SSRI Use",5,1 ],
    93: ["SSRIs for Depression in Cancer", 1],
    94: ["Canine Behavioral Disorders", 0], # agression in dogs mostly
    95: ["Transcranial Direct Current Stimulation (tDCS) for Depression", 1],
    96: ["Gatways to Clinical Trials", 5 ],
    97: ["SRRIs and Ocular Safety",1 ], # mostly glaucoma
    98: ["Folate for Depression", 1], # other nutrition topics?
    #####################################
    ### from here on under 100 papers ###
    #####################################
    99: ["Orofacial Pain Disorders", 3], #bruxism and burning mouth syndrome. treatment and safety -.-
    100: ["SSRIs for Depression in Kidney Disease Patients", 1], # 
    101: ["SSRIs for Enuresis", 1], # or enuresis or Stress Urinary Incontinence Or Urinary Function?
    102: ["St. John's Wort for Depression", 1], 
    103: ["SSRIs for Migraine", ], 
    104: ["SSRIs for Postpartum Depression", 1], 
    105: ["Behavioral Addictions",1 ], #  pathological gambling mostly, and problematic internet use
    106: ["Generic SSRIs", 1], # which category?
    107: ["SSRIs in Model Organisms ( C. Elegans and Drosophilia)", 2], 
    108: ["Differential Reinforcement of Low Rate (DRL) Operant Schedule", 2], 
    109: ["SSRIs for Dysthymia", 1], 
    110: ["SSRIs for Trichotillomania", 1], 
    111: ["Selective serotonin reuptake inhibitors",5,0 ], 
    112: ["Modulation of Serotonin Eelease by Autoreceptors and Heteroreceptors",2 ], 
    113: ["SSRIs and Galanin in Depression", 2], 
    114: ["SSRIs for Huntington Disease", 1], 
    115: ["SSRIs for Depression in and Treatment of Multiple Sclerosis", 1],  
    116: ["SSRIs for Depression in Chronic Obstructive Pulmonary Disease (COPD) Patients",1 ], 
    117: ["Quantification of Dopamine and Serotonin Release and Uptake",2 ], 
    118: ["Novel Compounds Synthesis and Antidepressant Potential", 2], 
    119: ["Thyroid Hormone for Depression", 1], 
    120: ["Olfactory Bbulbectomized (OB) Rat Model for Depression", 1], 
    121: ["SSRIs for Depression in HIV-positive Patients", ], 
    122: ["SSRIs Effect on Brain-derived Neurotrophic Factor (BDNF) Levels in Depressive Patients", 1], # may this be a biomarker?
    123: ["Paroxetines Therapeutic Potential as G Protein-coupled Receptor Kinase 2 (GRK2) Inhibitor", 3], # for cardiac diseases
    124: ["SSRIs for Smoking Cessation", 1], 
    125: ["Loudness Dependence of Auditory Evoked Potentials (LDAEP) as Biomarker for Depression", 2], 
    126: ["SSRIs for Syncope", 1], 
    127: ["SSRIs for Tinnitus ",1], 
    ####################################
    ### from here on under 50 papers ###
    ####################################
    128: ["Exercise for Depression", ], 
    129: ["SSRIs for Narcolepsy", 1], 
    130: ["Risk of Hallucinations",1 ], 
    131: ["SSRIs for Congenital Myasthenic Syndrome (SCCMS)", 1], 
    132: ["Tryptophan Depletion in Depression", 1], 
    133: ["SSRIs for Tourette's Syndrome", ], 
    134: ["Adulterants in Weight Loss Supplements", 0], # fluoxetine 
    135: ["Risk of Microscopic Colitis", 1], 
    136: ["Critical Psychiatry", 0], 
    137: ["Animal Models of Obsessive-compulsive Disorder (OCD) ", 2], 
    138: ["Psychotic Depression (STOP-PD Study)", 0, 1 ], 
    139: ["SSRIs for Pediatric OCD", ], 
    ####################################
    ### from here on under 25 papers ###
    ####################################
    140: ["Treatment of Paroxetine Induced Sexual Dysfunction",2 ], 
    141: ["SSRIs and Bariatric Surgery", ], #antiobesity surgery
    142: ["Methylphenidate augmentation in late-life depression", 1],
    143: ["Psilocybin for Depression",1 ], 
    144: ["Risk of Priapism", 1], #painful erection 
    145: ["SSRIs for Depersonalization Disorder",1 ], 
    ####################################
    ### from here on under 19 papers ###
    ####################################
    146: ["Serotonin Modulation Effects on Primates and Rodents", 2,5],
    147: ['Risk of Reversible Rerebral Vasoconstriction Syndrome (RCVS)',1 ],
    148: ["Gateways to Clinical Trials",5 ], 
    149: ["Citalopram Effect on Indoleamine Metabolism in Maturing Mouse Brains", 2,5], 
    150: ["Depressed Infertile Women", 5], 
    151: ["SSRIs for Olfactory reference syndrome (ORS)", 5], 
}

path = f"../output/cluster-qualifications/K10/pythonic_cluster_labels_{params}.json"

with open(path, "w") as f:
    json.dump(cluster_labels, f)

In [6]:
import json

json_string = """
{
    "0": { "label": "Serotonin Receptor Studies", "evidencetype": [2] },
    "1": { "label": "Aquatic Ecotoxicology", "evidencetype": [0] },
    "2": { "label": "Prenatal Exposure", "evidencetype": [1, 2] },
    "3": {
        "label": "Quantification of SSRIs in Biological Samples",
        "evidencetype": [2]
    },
    "4": {
        "label": "Treatment of Obsessive-Compulsive Disorder (OCD)",
        "evidencetype": [1]
    },
    "5": { "label": "SSRIs and the Cytochrome P450 System", "evidencetype": [2] },
    "6": { "label": "Neurogenesis", "evidencetype": [2] },
    "7": { "label": "Pediatric Depression", "evidencetype": [1] },
    "8": {
        "label": "Herbal and Natural Compounds for Depression, using Forced Swimming Test",
        "evidencetype": [2]
    },
    "9": { "label": "Paroxetine Binding", "evidencetype": [2] },
    "10": { "label": "Neural Processing of Emotional Cues", "evidencetype": [1] },
    "11": { "label": "Prenatal Exposure", "evidencetype": [2] },
    "12": { "label": "Forced Swimming Test", "evidencetype": [2] },
    "13": {
        "label": "Serotonin Transporter Gene and Antidepressant Response",
        "evidencetype": [1]
    },
    "14": { "label": "Fluvoxamine for Depression", "evidencetype": [2] },
    "15": { "label": "SSRIs for PTSD", "evidencetype": [2] },
    "16": { "label": "Sexual Dysfunction", "evidencetype": [1] },
    "17": { "label": "Sequenced Depression Treatment", "evidencetype": [1] },
    "18": { "label": "Serotonin Syndrome", "evidencetype": [1] },
    "19": { "label": "Serotonin and Inflammation", "evidencetype": [3] },
    "20": { "label": "Post-Stroke SSRI Use", "evidencetype": [1] },
    "21": {
        "label": "Depression and Cardiovascular Pathology",
        "evidencetype": [1]
    },
    "22": { "label": "SSRIs for Pain", "evidencetype": [1] },
    "23": { "label": "SSRIs for Bipolar Depression", "evidencetype": [1] },
    "24": { "label": "SSRI Utilization Patterns", "evidencetype": [1] },
    "25": { "label": "Escitalopram for Depression", "evidencetype": [1] },
    "26": { "label": "SSRIs in Dementias", "evidencetype": [1] },
    "27": { "label": "Bleeding Risk", "evidencetype": [1] },
    "28": { "label": "Serotonin Receptor Studies", "evidencetype": [2] },
    "29": { "label": "SSRIs for Weight Management", "evidencetype": [1] },
    "30": { "label": "Suicide Risk", "evidencetype": [1] },
    "31": { "label": "Cost-Effectiveness", "evidencetype": [0] },
    "32": { "label": "Conditioned Fear", "evidencetype": [2] },
    "33": { "label": "Paroxetine", "evidencetype": [4, 5] },
    "34": { "label": "SSRIs for Panic Disorder", "evidencetype": [1] },
    "35": {
        "label": "Herbal and Natural Compounds for Depression",
        "evidencetype": [2]
    },
    "36": { "label": "Fluvoxamine for Covid 19", "evidencetype": [1] },
    "37": {
        "label": "SSRIs for Generalized Anxiety Disorder (GAD)",
        "evidencetype": [1]
    },
    "38": { "label": "Fracture Risk", "evidencetype": [1] },
    "39": { "label": "Fluoxetine", "evidencetype": [4, 5] },
    "40": { "label": "Extrapyramidal Reactions Risk", "evidencetype": [1] },
    "41": {
        "label": "SSRIs Effect on Sleep in Depressed Patients",
        "evidencetype": [1]
    },
    "42": { "label": "Hot Flushes (Menopause)", "evidencetype": [1] },
    "43": { "label": "SSRIs for Dermatological Diseases", "evidencetype": [1] },
    "44": {
        "label": "Oncopreventive and Oncolytic Properties of SSRIs",
        "evidencetype": [1]
    },
    "45": { "label": "Premature Ejaculation", "evidencetype": [1] },
    "46": { "label": "QTC Prolongation", "evidencetype": [1] },
    "47": {
        "label": "SSRIs for Premenstrual Dysphoric Disorder (PMDD)",
        "evidencetype": [1]
    },
    "48": { "label": "Depression in Diabetes", "evidencetype": [1] },
    "49": {
        "label": "SSRIs for Depression with Cognitive Impairment",
        "evidencetype": [1]
    },
    "50": { "label": "SSRIs for Eating Disorders", "evidencetype": [1] },
    "51": { "label": "SSRIs for Alcohol Dependence", "evidencetype": [1] },
    "52": { "label": "SSRIs for Irritable Bowel Disease", "evidencetype": [1] },
    "53": {
        "label": "Personality Disorder; Learning; Behavioral Effects; Introception",
        "evidencetype": [4]
    },
    "54": { "label": "Depression Treatment Review", "evidencetype": [1, 4] },
    "55": { "label": "Depression in Epileptic Patients", "evidencetype": [1] },
    "56": { "label": "SSRIs for Autism Spectrum Disorders", "evidencetype": [1] },
    "57": { "label": "(Sertraline) Drug Delivery", "evidencetype": [2] },
    "58": { "label": "Depression in Parkinson's Patients", "evidencetype": [] },
    "59": { "label": "Hepatotoxic Risk", "evidencetype": [2] },
    "60": { "label": "SSRI Synthesis", "evidencetype": [2] },
    "61": {
        "label": "SSRIs (Fluvoxamine) for Schizophrenia",
        "evidencetype": [1]
    },
    "62": { "label": "SSRIs for Substance Abuse (Cocaine)", "evidencetype": [2] },
    "63": {
        "label": "SSRIs for Social Anxiety Disorder",
        "evidencetype": [1, 4]
    },
    "64": { "label": "Sertraline for Depression", "evidencetype": [1] },
    "65": { "label": "Drug-Drug Interactions", "evidencetype": [1] },
    "66": { "label": "Ion Channels", "evidencetype": [2] },
    "67": {
        "label": "Methylenedioxymethamphetamine (MDMA) Effects on Serotonin",
        "evidencetype": [2]
    },
    "68": { "label": "Genotype Mediated Response to SSRIs", "evidencetype": [2] },
    "69": { "label": "Hyponatremia Risk", "evidencetype": [1] },
    "70": {
        "label": "Neuroendocrine Effects of Citalopram",
        "evidencetype": [3]
    },
    "71": {
        "label": "SSRIs for Body Dysmorphic Disorder (BDD)",
        "evidencetype": [1]
    },
    "72": { "label": "Pharmacological Reviews", "evidencetype": [2, 4] },
    "73": { "label": "Forensic Drug Concentration", "evidencetype": [1] },
    "74": { "label": "Citalopram for Depression", "evidencetype": [1, 4] },
    "75": { "label": "SSRIs for Pediatric Anxiety", "evidencetype": [] },
    "76": { "label": "SSRI Safety Reviews", "evidencetype": [1, 4] },
    "77": { "label": "Pulmonary Hypertension", "evidencetype": [2] },
    "78": { "label": "SSRIs as Antifungal Agents", "evidencetype": [2] },
    "79": { "label": "Geriatric Depression", "evidencetype": [1, 4] },
    "80": { "label": "Withdrawal Symptoms", "evidencetype": [1] },
    "81": { "label": "Agomelatine for Depression", "evidencetype": [4] },
    "82": { "label": "SSRI Efficacy in Depression", "evidencetype": [1, 4] },
    "83": { "label": "Wastewater Management", "evidencetype": [0] },
    "84": { "label": "Astrocyte Receptors", "evidencetype": [1] },
    "85": { "label": "Sertraline", "evidencetype": [5] },
    "86": { "label": "Vilazodone for Depression", "evidencetype": [1] },
    "87": { "label": "Depression in Hepatitis Patients", "evidencetype": [1] },
    "88": {
        "label": "SSRIs for Depression after Traumatic Brain Injury",
        "evidencetype": [1]
    },
    "89": { "label": "SSRIs Effects on Dopamine Receptors", "evidencetype": [2] },
    "90": { "label": "Hyperprolactinemia-related Symptoms", "evidencetype": [2] }, 
    "91": { "label": "SSRIs for Fibromyalgia", "evidencetype": [2] },
    "92": { "label": "SSRI Use", "evidencetype": [5, 1] },
    "93": { "label": "SSRIs for Depression in Cancer", "evidencetype": [1] },
    "94": { "label": "Canine Behavioral Disorders", "evidencetype": [0] }, 
    "95": {
        "label": "Transcranial Direct Current Stimulation (tDCS) for Depression",
        "evidencetype": [1]
    },
    "96": { "label": "Gatways to Clinical Trials", "evidencetype": [5] },
    "97": { "label": "SRRIs Ocular Safety Risk", "evidencetype": [1] },
    "98": { "label": "Folate for Depression", "evidencetype": [1] },
  
    "99": { "label": "Orofacial Pain Disorders", "evidencetype": [3] },
    "100": {
        "label": "SSRIs for Depression in Kidney Disease Patients",
        "evidencetype": [1]
    },
    "101": { "label": "SSRIs for Enuresis", "evidencetype": [1] },
    "102": { "label": "St. John's Wort for Depression", "evidencetype": [1] },
    "103": { "label": "SSRIs for Migraine", "evidencetype": [1] },
    "104": { "label": "SSRIs for Postpartum Depression", "evidencetype": [1] },
    "105": { "label": "Behavioral Addictions", "evidencetype": [1] },
    "106": { "label": "Generic SSRIs", "evidencetype": [1] }, 
    "107": {
        "label": "SSRIs in Model Organisms ( C. Elegans and Drosophilia)",
        "evidencetype": [2]
    },
    "108": {
        "label": "Differential Reinforcement of Low Rate (DRL) Operant Schedule",
        "evidencetype": [2]
    },
    "109": { "label": "SSRIs for Dysthymia", "evidencetype": [1] },
    "110": { "label": "SSRIs for Trichotillomania", "evidencetype": [1] },
    "111": {
        "label": "Selective Serotonin Reuptake Inhibitors",
        "evidencetype": [5, 0]
    },
    "112": {
        "label": "Modulation of Serotonin Release by Autoreceptors and Heteroreceptors",
        "evidencetype": [2]
    },
    "113": { "label": "SSRIs and Galanin in Depression", "evidencetype": [2] },
    "114": { "label": "SSRIs for Huntington Disease", "evidencetype": [1] },
    "115": {
        "label": "SSRIs for Depression in and Treatment of Multiple Sclerosis",
        "evidencetype": [1]
    },
    "116": {
        "label": "SSRIs for Depression in Chronic Obstructive Pulmonary Disease (COPD) Patients",
        "evidencetype": [1]
    },
    "117": {
        "label": "Quantification of Dopamine and Serotonin Release and Uptake",
        "evidencetype": [2]
    },
    "118": {
        "label": "Novel Compounds Synthesis and Antidepressant Potential",
        "evidencetype": [2]
    },
    "119": { "label": "Thyroid Hormone for Depression", "evidencetype": [1] },
    "120": {
        "label": "Olfactory Bbulbectomized (OB) Rat Model for Depression",
        "evidencetype": [1]
    },
    "121": {
        "label": "SSRIs for Depression in HIV-positive Patients",
        "evidencetype": [1]
    },
    "122": {
        "label": "SSRIs Effect on Brain-derived Neurotrophic Factor (BDNF) Levels in Depressive Patients",
        "evidencetype": [1]
    }, 
    "123": {
        "label": "Paroxetines Therapeutic Potential as G Protein-coupled Receptor Kinase 2 (GRK2) Inhibitor",
        "evidencetype": [3]
    },
    "124": { "label": "SSRIs for Smoking Cessation", "evidencetype": [1] },
    "125": {
        "label": "Loudness Dependence of Auditory Evoked Potentials (LDAEP) as Biomarker for Depression",
        "evidencetype": [2]
    },
    "126": { "label": "SSRIs for Syncope", "evidencetype": [1] },
    "127": { "label": "SSRIs for Tinnitus", "evidencetype": [1] },
    "128": { "label": "Exercise for Depression", "evidencetype": [1] },
    "129": { "label": "SSRIs for Narcolepsy", "evidencetype": [1] },
    "130": { "label": "Risk of Hallucinations", "evidencetype": [1] },
    "131": {
        "label": "SSRIs for Congenital Myasthenic Syndrome (SCCMS)",
        "evidencetype": [1]
    },
    "132": { "label": "Tryptophan Depletion in Depression", "evidencetype": [1] },
    "133": { "label": "SSRIs for Tourette's Syndrome", "evidencetype": [1] },
    "134": {
        "label": "Adulterants in Weight Loss Supplements",
        "evidencetype": [0]
    }, 
    "135": { "label": "Risk of Microscopic Colitis", "evidencetype": [1] },
    "136": { "label": "Critical Psychiatry", "evidencetype": [0] },
    "137": {
        "label": "Animal Models of Obsessive-compulsive Disorder (OCD)",
        "evidencetype": [2]
    },
    "138": {
        "label": "Psychotic Depression (STOP-PD Study)",
        "evidencetype": [1]
    },
    "139": { "label": "SSRIs for Pediatric OCD", "evidencetype": [1] },
    "140": {
        "label": "Treatment of Paroxetine Induced Sexual Dysfunction",
        "evidencetype": [2]
    },
    "141": { "label": "SSRIs and Bariatric Surgery", "evidencetype": [1] },
    "142": {
        "label": "Methylphenidate augmentation in late-life depression",
        "evidencetype": [1]
    },
    "143": { "label": "Psilocybin for Depression", "evidencetype": [1] },
    "144": { "label": "Risk of Priapism", "evidencetype": [1] }, 
    "145": {
        "label": "SSRIs for Depersonalization Disorder",
        "evidencetype": [1]
    },
    "146": {    
        "label": "Serotonin Modulation Effects on Primates and Rodents",
        "evidencetype":[2, 5]
    },
    "147": {
        "label": "Risk of Reversible Rerebral Vasoconstriction Syndrome (RCVS)",
        "evidencetype": [1]
    },
    "148": { "label": "Gateways to Clinical Trials", "evidencetype": [5] },
    "149": {
        "label": "Citalopram Effect on Indoleamine Metabolism in Maturing Mouse Brains",
        "evidencetype": [2, 5]
    },
    "150": { "label": "Depressed Infertile Women", "evidencetype": [5] },
    "151": {
        "label": "SSRIs for Olfactory reference syndrome (ORS)",
        "evidencetype": [5]
    }
}
"""

cluster_labels = json.loads(json_string)


# Add info and turn into excel sheet


In [7]:
# read summary sheet from output/tables/cluster-explorer/FinalSelect/SingleSolExplorer_alpha0.3_k10_res0.002.xlsx

df_summary = pd.read_excel("../output/tables/cluster-explorer/FinalSelect/SingleSolExplorer_alpha0.3_k10_res0.002.xlsx", sheet_name="summary")
df_summary['Cluster'] = df_summary['Cluster'].astype(int)
df_summary.head(2)

,Cluster,Nr of Pubs,25th Percentile Year,Median Year,75th Percentile Year,label,Word_0,Word_1,Word_2,Word_3,...,Word_10,Word_11,Word_12,Word_13,Word_14,Word_15,Word_16,Word_17,Word_18,Word_19
0,0,1198,1996.0,2001.0,2008.0,Serotonin Receptor Studies,ht,fluoxetine,receptor,serotonin,...,citalopram,levels,effect,effects,rats,antidepressant,extracellular,binding,inf,1a
1,1,1092,2014.0,2018.0,2021.0,Aquatic Ecotoxicology,fluoxetine,pharmaceuticals,fish,effects,...,water,zebrafish,treatment,serotonin,aquatic,μg,sertraline,study,exposed,removal


In [8]:
def process_cluster_data(cluster_labels, df_summary):
    # Create DataFrame from cluster_labels dictionary
    df_clusters = pd.DataFrame(cluster_labels).T
    df_clusters['cluster'] = df_clusters.index.astype(int)

    # Add columns for each evidence type
    evidence_types = {
        "clinical": 1,
        "in_vivo_in_vitro": 2,
        "mixed": 3,
        "review": 4,
        "garbage": 5
    }
    for col_name, evid_type in evidence_types.items():
        df_clusters[col_name] = df_clusters["evidencetype"].apply(lambda x: 1 if evid_type in x else 0)

    # Remove the evidencetype column
    df_clusters.drop(columns="evidencetype", inplace=True)

    # Merge with df_summary on cluster and Cluster columns
    df_clusters = df_clusters.merge(
        df_summary[['Cluster', 'Nr of Pubs', '25th Percentile Year', 'Median Year', '75th Percentile Year']],
        left_on="cluster",
        right_on="Cluster",
        how="left"
    )

    # Drop the Cluster column
    df_clusters.drop(columns="Cluster", inplace=True)

    # Make all column names lowercase and replace spaces with underscores
    df_clusters.columns = [col.lower().replace(' ', '_') for col in df_clusters.columns]

    return df_clusters

# Use the function
df_cluster_info = process_cluster_data(cluster_labels, df_summary)
df_cluster_info.head(2)

,label,cluster,clinical,in_vivo_in_vitro,mixed,review,garbage,nr_of_pubs,25th_percentile_year,median_year,75th_percentile_year
0,Serotonin Receptor Studies,0,0,1,0,0,0,1198,1996.0,2001.0,2008.0
1,Aquatic Ecotoxicology,1,0,0,0,0,0,1092,2014.0,2018.0,2021.0


# add categories to it


In [9]:
json_string = """
{
  "Pharmacology": {
    "Pharmacodynamics": {
      "Mechanism of action": [0, 2, 28,84, 89, 112], 
      "Animal Models of Disorders": [12, 32, 108, 120, 137],
      "OtherPharmacodynamics": [6, 11, 66, 67, 78, 107, 123, 125, 140]
    },
    "Pharmacokinetics": [5, 57],
    "Pharmacogenetics": [13, 68],
    "Analytical Methods": [3, 117],
    "Biomarker Search": [10, 122],
    "OtherPharmacology": [2, 10, 60,70, 72, 118]
  },
  "Indications": {
    "Disorders": {
      "Depression": [
        7, 8, 12, 13, 14, 17, 21, 25, 41, 48, 54, 55, 58, 64, 74, 79, 81, 82,
        86, 87, 88, 93, 95, 98, 100, 102, 113, 115, 116, 119, 120, 121, 122,
        125, 128, 132, 138, 141, 142, 23, 47, 19, 41
      ],
      "Anxiety": [32, 37, 63, 75],
      "OCD": [4, 137, 139],
      "Weight and Eating Disorders": [29, 50],
      "Substance Abuse": [51, 124], 
      "OtherDisorders": [
        15, 22, 34, 36, 42, 43, 45, 52, 56, 61, 62, 71, 77,78, 91, 99, 101, 103,
        104, 105, 109, 110, 114, 115, 123, 126, 127, 129, 133, 144
      ]
    },
    "Special Populations": {
      "Pediatrics": [7, 56, 75, 139],
      "Geriatrics": [26, 38, 49, 58, 65, 69, 79, 115, 141],
      "OtherSpecialPop": [20, 21, 48, 55, 87, 88, 93, 100, 116, 121, 131]
    },
    "OtherIndications": []
  },
  "Safety": {
    "Prenatal Exposure": [2, 11],
    "Pregnancy": [2],
    "Suicide": [7, 30],
    "Sexual Dysfunction": [16, 140, 143],
    "Toxicity": [18, 59],
    "Bleeding Risk": [27],
    "Cardiovascular": [46],
    "OtherSafety": [38, 40, 44, 65, 69, 73, 76, 80, 90, 97, 99,106, 130, 135, 147]
  },
  "OtherRoot": {
    "Drugs": {
      "Paroxetine": [9, 33, 140],
      "Fluoxetine": [39, 134],
      "Fluoxamine": [14],
      "Escitalopram": [25],
      "Sertraline": [64, 85],
      "Agomelatine": [81],
      "Vilazodone": [86],
      "Psilocybin": [142]
    },
    "Ecotoxicology": [1, 83],
    "Alternative Treatments": [8, 35],
    "Drug Utilization Patterns": [24,106],
    "Cost-Effectineness": [31],
    "Other": [53,92, 94, 96, 105, 111, 134, 136, 145, 146, 147, 148,149,150,151]
  }
}
"""

tree_hierachy = json.loads(json_string)

In [10]:
# Load the JSON string
tree_hierarchy = json.loads(json_string)

# Function to convert the hierarchy into a list of dictionaries
def flatten_hierarchy(hierarchy, path=None):
    if path is None:
        path = []
    if isinstance(hierarchy, dict):
        result = []
        for key, value in hierarchy.items():
            result.extend(flatten_hierarchy(value, path + [key]))
        return result
    else:
        return [{**{level: 1 for level in path}, 'cluster': cluster} for cluster in hierarchy]

# Flatten the hierarchy
flattened_data = flatten_hierarchy(tree_hierarchy)

# Convert to DataFrame
df_hierarchy = pd.DataFrame(flattened_data).explode('cluster').reset_index(drop=True)

# Replace NaN with 0 for missing hierarchy levels
df_hierarchy = df_hierarchy.fillna(0)

# Convert the 'cluster' column to int for merging
df_hierarchy['cluster'] = df_hierarchy['cluster'].astype(int)

print('nr unique clusters: ', df_hierarchy['cluster'].nunique())
print('Entries: ', df_hierarchy.shape[0])

df_hierarchy = df_hierarchy.groupby('cluster').max().reset_index()
print('Entries after grouping: ', df_hierarchy.shape[0])

print('Total Number of branches: ', df_hierarchy.shape[1] - 1) 

df_hierarchy.head(2)

nr unique clusters:  152
Entries:  203
Entries after grouping:  152
Total Number of branches:  46


,cluster,Pharmacology,Pharmacodynamics,Mechanism of action,Animal Models of Disorders,OtherPharmacodynamics,Pharmacokinetics,Pharmacogenetics,Analytical Methods,Biomarker Search,...,Escitalopram,Sertraline,Agomelatine,Vilazodone,Psilocybin,Ecotoxicology,Alternative Treatments,Drug Utilization Patterns,Cost-Effectineness,Other
0,0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [11]:
# merge on cluster with df_cluster_info
df_cluster_info = df_cluster_info.merge(df_hierarchy, on='cluster', how='left')
print(df_cluster_info.shape)
df_cluster_info.tail()

(152, 57)


,label,cluster,clinical,in_vivo_in_vitro,mixed,review,garbage,nr_of_pubs,25th_percentile_year,median_year,...,Escitalopram,Sertraline,Agomelatine,Vilazodone,Psilocybin,Ecotoxicology,Alternative Treatments,Drug Utilization Patterns,Cost-Effectineness,Other
147,Risk of Reversible Rerebral Vasoconstriction S...,147,1,0,0,0,0,15,1996.50,2000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
148,Gateways to Clinical Trials,148,0,0,0,0,1,15,2010.00,2015.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
149,Citalopram Effect on Indoleamine Metabolism in...,149,0,1,0,0,1,11,2004.00,2005.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
150,Depressed Infertile Women,150,0,0,0,0,1,8,1986.00,1987.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
151,SSRIs for Olfactory reference syndrome (ORS),151,0,0,0,0,1,8,2010.25,2013.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
p = '/Users/jlq293/Projects/Study-1-Bibliometrics/output/cluster-qualifications/K10/'
df_cluster_info.to_excel(p + 'ClusterInfoLabelTree.xlsx', index=False)

In [13]:
df_cluster_info.columns

Index(['label', 'cluster', 'clinical', 'in_vivo_in_vitro', 'mixed', 'review',
       'garbage', 'nr_of_pubs', '25th_percentile_year', 'median_year',
       '75th_percentile_year', 'Pharmacology', 'Pharmacodynamics',
       'Mechanism of action', 'Animal Models of Disorders',
       'OtherPharmacodynamics', 'Pharmacokinetics', 'Pharmacogenetics',
       'Analytical Methods', 'Biomarker Search', 'OtherPharmacology',
       'Indications', 'Disorders', 'Depression', 'Anxiety', 'OCD',
       'Weight and Eating Disorders', 'Substance Abuse', 'OtherDisorders',
       'Special Populations', 'Pediatrics', 'Geriatrics', 'OtherSpecialPop',
       'Safety', 'Prenatal Exposure', 'Pregnancy', 'Suicide',
       'Sexual Dysfunction', 'Toxicity', 'Bleeding Risk', 'Cardiovascular',
       'OtherSafety', 'OtherRoot', 'Drugs', 'Paroxetine', 'Fluoxetine',
       'Fluoxamine', 'Escitalopram', 'Sertraline', 'Agomelatine', 'Vilazodone',
       'Psilocybin', 'Ecotoxicology', 'Alternative Treatments',
       'D

# Build Full Tree Incl Labels


In [16]:
cluster_label_dict = dict(zip(df_cluster_info.cluster, df_cluster_info.label))

{0: 'Serotonin Receptor Studies',
 1: 'Aquatic Ecotoxicology',
 2: 'Prenatal Exposure',
 3: 'Quantification of SSRIs in Biological Samples',
 4: 'Treatment of Obsessive-Compulsive Disorder (OCD)',
 5: 'SSRIs and the Cytochrome P450 System',
 6: 'Neurogenesis',
 7: 'Pediatric Depression',
 8: 'Herbal and Natural Compounds for Depression, using Forced Swimming Test',
 9: 'Paroxetine Binding',
 10: 'Neural Processing of Emotional Cues',
 11: 'Prenatal Exposure',
 12: 'Forced Swimming Test',
 13: 'Serotonin Transporter Gene and Antidepressant Response',
 14: 'Fluvoxamine for Depression',
 15: 'SSRIs for PTSD',
 16: 'Sexual Dysfunction',
 17: 'Sequenced Depression Treatment',
 18: 'Serotonin Syndrome',
 19: 'Serotonin and Inflammation',
 20: 'Post-Stroke SSRI Use',
 21: 'Depression and Cardiovascular Pathology',
 22: 'SSRIs for Pain',
 23: 'SSRIs for Bipolar Depression',
 24: 'SSRI Utilization Patterns',
 25: 'Escitalopram for Depression',
 26: 'SSRIs in Dementias',
 27: 'Bleeding Risk',
 2

In [19]:
# Define a function to replace numbers with labels
def replace_numbers_with_labels(node, mapping):
    if isinstance(node, list):
        return [mapping.get(item, item) for item in node]
    elif isinstance(node, dict):
        return {key: replace_numbers_with_labels(value, mapping) for key, value in node.items()}
    else:
        return node

# Replace numbers with labels in the hierarchy
updated_tree_hierarchy = replace_numbers_with_labels(tree_hierarchy, cluster_label_dict)

# Print the updated hierarchy to verify the changes
print(json.dumps(updated_tree_hierarchy, indent=2))

# save as json 
with open(p + 'FullClusterLabelTree.json', 'w') as f:
    json.dump(updated_tree_hierarchy, f, indent=2)

{
  "Pharmacology": {
    "Pharmacodynamics": {
      "Mechanism of action": [
        "Serotonin Receptor Studies",
        "Prenatal Exposure",
        "Serotonin Receptor Studies",
        "Astrocyte Receptors",
        "SSRIs Effects on Dopamine Receptors",
        "Modulation of Serotonin Release by Autoreceptors and Heteroreceptors"
      ],
      "Animal Models of Disorders": [
        "Forced Swimming Test",
        "Conditioned Fear",
        "Differential Reinforcement of Low Rate (DRL) Operant Schedule",
        "Olfactory Bbulbectomized (OB) Rat Model for Depression",
        "Animal Models of Obsessive-compulsive Disorder (OCD)"
      ],
      "OtherPharmacodynamics": [
        "Neurogenesis",
        "Prenatal Exposure",
        "Ion Channels",
        "Methylenedioxymethamphetamine (MDMA) Effects on Serotonin",
        "SSRIs as Antifungal Agents",
        "SSRIs in Model Organisms ( C. Elegans and Drosophilia)",
        "Paroxetines Therapeutic Potential as G Protein-co

In [21]:
import json
from graphviz import Digraph


In [26]:
import json
from graphviz import Digraph

# Function to create a graph from the hierarchy
def add_nodes_edges(graph, hierarchy, parent=None):
    for key, value in hierarchy.items():
        # Create a node for the current key
        graph.node(key)
        # Add an edge from the parent to the current node
        if parent:
            graph.edge(parent, key)
        # If the value is a dict, recurse
        if isinstance(value, dict):
            add_nodes_edges(graph, value, key)
        # If the value is a list, add the items as nodes
        elif isinstance(value, list):
            for item in value:
                graph.node(item)
                graph.edge(key, item)

# Function to plot the hierarchy with adjustable figure size
def plot_hierarchy(hierarchy, width="8", height="6"):
    dot = Digraph(graph_attr={"size": f"{width},{height}"})
    add_nodes_edges(dot, hierarchy)
    return dot

# Example JSON hierarchy
json_string = """
{
  "Pharmacology": {
    "Pharmacodynamics": {
      "Mechanism of action": ["Label_A", "Label_C", "Label_X", "Label_Y", "Label_Z"], 
      "Animal Models of Disorders": ["Label_M", "Label_N", "Label_O", "Label_P", "Label_Q"],
      "Other": ["Label_B", "Label_D", "Label_E", "Label_F", "Label_G", "Label_H", "Label_I", "Label_J"]
    },
    "Pharmacokinetics": ["Label_K", "Label_L"],
    "Pharmacogenetics": ["Label_R", "Label_S"],
    "Analytical Methods": ["Label_T", "Label_U"],
    "Biomarker Search": ["Label_V", "Label_W"],
    "Other": ["Label_A", "Label_V", "Label_60", "Label_72", "Label_118"]
  },
  "Indications": {
    "Disorders": {
      "Depression": [
        "Label_7", "Label_8", "Label_12", "Label_13", "Label_14", "Label_17", "Label_21", "Label_25", "Label_41", "Label_48", "Label_54", "Label_55", "Label_58", "Label_64", "Label_74", "Label_79", "Label_81", "Label_82",
        "Label_86", "Label_87", "Label_88", "Label_93", "Label_95", "Label_98", "Label_100", "Label_102", "Label_113", "Label_115", "Label_116", "Label_119", "Label_120", "Label_121", "Label_122",
        "Label_125", "Label_128", "Label_132", "Label_138", "Label_141", "Label_142", "Label_23", "Label_47", "Label_19", "Label_41"
      ],
      "Anxiety": ["Label_32", "Label_37", "Label_63", "Label_75"],
      "OCD": ["Label_4", "Label_137", "Label_139"],
      "Weight and Eating Disorders": ["Label_29", "Label_50"],
      "Substance Abuse": ["Label_51", "Label_124"], 
      "Other": [
        "Label_15", "Label_22", "Label_34", "Label_36", "Label_42", "Label_43", "Label_45", "Label_52", "Label_56", "Label_61", "Label_62", "Label_71", "Label_78", "Label_91", "Label_99", "Label_101", "Label_103",
        "Label_104", "Label_105", "Label_109", "Label_110", "Label_114", "Label_115", "Label_123", "Label_126", "Label_127", "Label_129", "Label_133", "Label_144"
      ]
    },
    "Special Populations": {
      "Pediatrics": ["Label_7", "Label_56", "Label_75", "Label_139"],
      "Geriatrics": ["Label_26", "Label_38", "Label_49", "Label_58", "Label_65", "Label_69", "Label_79", "Label_115", "Label_141"],
      "Other": ["Label_20", "Label_21", "Label_48", "Label_55", "Label_87", "Label_88", "Label_93", "Label_100", "Label_116", "Label_121", "Label_131"]
    },
    "Other": []
  },
  "Safety": {
    "Prenatal Exposure": ["Label_2", "Label_11"],
    "Pregnancy": ["Label_2"],
    "Suicide": ["Label_7", "Label_1"],
    "Sexual Dysfunction": ["Label_0", "Label_140", "Label_143"],
    "Toxicity": ["Label_18", "Label_59"],
    "Bleeding Risk": ["Label_27"],
    "Cardiovascular": ["Label_46"],
    "Other": ["Label_38", "Label_40", "Label_44", "Label_65", "Label_69", "Label_73", "Label_76", "Label_80", "Label_90", "Label_97", "Label_99", "Label_130", "Label_135", "Label_147"]
  },
  "Other": {
    "Drugs": {
      "Paroxetine": ["Label_9", "Label_33", "Label_140"],
      "Fluoxetine": ["Label_39", "Label_134"],
      "Fluoxamine": ["Label_14"],
      "Escitalopram": ["Label_25"],
      "Sertraline": ["Label_64", "Label_85"],
      "Agomelatine": ["Label_81"],
      "Vilazodone": ["Label_86"],
      "Psilocybin": ["Label_142"]
    },
    "Ecotoxicology": ["Label_1", "Label_83"],
    "Alternative Treatments": ["Label_8", "Label_35"],
    "Drug Utilization Patterns": ["Label_24"],
    "Cost-Effectiveness": ["Label_31"],
    "Other": ["Label_53", "Label_94", "Label_96", "Label_105", "Label_111", "Label_134", "Label_136", "Label_145", "Label_147"]
  }
}
"""

tree_hierarchy = json.loads(json_string)

# Plot the hierarchy with a specified size
dot = plot_hierarchy(tree_hierarchy, width="100", height="800")
dot = plot_hierarchy(tree_hierarchy, width="10", height="8", dpi=300)


TypeError: plot_hierarchy() got an unexpected keyword argument 'dpi'